## Задача из трека по ИИ RuCodeFestival 2020 

В этом конкурсе необходимо решить задачу предсказания цены подержанной иномарки, используя характеристики машины.

Обратите внимание, что данные могут содержать неверно спарсенные поля.

Метрика оценивания -- r^2-score

https://www.kaggle.com/c/rucode-carprice/data

In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Загрузим данные и посмотрим на них

In [28]:
data_train = pd.read_csv('rucode-carprice/data_train.csv')
data_test = pd.read_csv('rucode-carprice/data_test.csv')

In [29]:
data_train.head()

,Unnamed: 0,Марка,Модель,Год выпуска,Рабочий объем,Мощность двигателя,Тип топлива,Трансмиссия,Количество мест,Количество дверей,Пробег,Цена
0,0,audi,a8,2000,3328.0,166.0,дизель,авто,5.0,4.0,352640.0,101800
1,1,nissan,juke,2013,NaN,81.0,дизель,ручной,NaN,4.0,63460.0,250200
2,2,skoda,fabia,2014,1199.0,55.0,NaN,ручной,5.0,5.0,15865.0,201700
3,3,audi,a3,2013,1968.0,110.0,NaN,ручной,NaN,4.0,44606.3,491300
4,4,bmw,NaN,1999,2793.0,142.0,бензин,ручной,5.0,4.0,187150.0,56100


In [30]:
data_train.shape

(146012, 12)

In [31]:
data_test.shape

(74016, 11)

In [32]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146012 entries, 0 to 146011
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          146012 non-null  int64  
 1   Марка               146012 non-null  object 
 2   Модель              110867 non-null  object 
 3   Год выпуска         146012 non-null  int64  
 4   Рабочий объем       133953 non-null  float64
 5   Мощность двигателя  146012 non-null  float64
 6   Тип топлива         55760 non-null   object 
 7   Трансмиссия         146012 non-null  object 
 8   Количество мест     122633 non-null  float64
 9   Количество дверей   135322 non-null  float64
 10  Пробег              146012 non-null  float64
 11  Цена                146012 non-null  int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 13.4+ MB


Заполним пропуски

In [33]:
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

Отберем числовые признаки 

In [34]:
num_list = ['Год выпуска','Мощность двигателя','Пробег','Рабочий объем','Количество мест','Количество дверей']

In [35]:
y = data_train['Цена']
X = data_train.loc[:,num_list]

Разделим выборку на train/test и обучим линейную регрессию

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.25)
lr = LinearRegression()

model = lr.fit(X_train, y_train)
print("R^2 is: ", model.score(X_test, y_test))

R^2 is:  0.12265044564563288


Добавим пару категориальных признаков и посмотрим, как изменится score

In [37]:
cat_list = ['Марка', 'Трансмиссия']

In [38]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [39]:
cat_train = data_train[cat_list]
cat_test = data_test[cat_list]
for c in cat_list:
    cat_train = create_dummies(cat_train, c)
    cat_test = create_dummies(cat_test, c)

In [40]:
X = pd.concat([data_train.loc[:,num_list],cat_train.drop(cat_list, axis=1)],axis=1)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.25)
lr = LinearRegression()

model = lr.fit(X_train, y_train)
print("R^2 is: ", model.score(X_test, y_test))

R^2 is:  0.1310746360670364


Получим предсказание на тестовой выборке и запишем результат в файл

In [42]:
clf = LinearRegression()
clf.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:
for i in cat_test.columns:
    if i not in cat_train.columns:
        cat_list.append(i)

In [46]:
submission = pd.DataFrame()
submission['Id'] = data_test.iloc[:,0]
submission['Price'] = clf.predict(pd.concat([data_test.loc[:,num_list], cat_test.drop(cat_list, axis=1)],axis=1))
submission.to_csv('baseline.csv', index=False)